In [ ]:
import networkx as nx
import random
from math import sqrt
import torch
from torch_geometric.utils.convert import from_networkx
import numpy as np
from python_tsp.exact import solve_tsp_dynamic_programming


In [ ]:
# TSP instance params
# num_nodes = int(random.uniform(20, 40))
# num_nodes = int(random.uniform(3, 5)) # for testing
num_nodes = 15 # for testing
nodes = range(num_nodes)
x_values = [random.uniform(0, sqrt(2)/2) for n in nodes]
y_values = [random.uniform(0, sqrt(2)/2) for n in nodes]
print(f"num_nodes: {num_nodes}")

In [ ]:
# create TSP instance graph
g = nx.Graph()
for node in nodes:
    g.add_node(node, x=x_values[node], y=y_values[node])
for src_node_id in nodes:
    for dst_node_id in nodes:
        if(src_node_id != dst_node_id ):
            x1 = g.nodes[src_node_id]["x"]
            y1 = g.nodes[src_node_id]["y"]
            x2 = g.nodes[dst_node_id]["x"]
            y2 = g.nodes[dst_node_id]["y"]
            euclidian_distance = sqrt((x1-x2)**2 + (y1-y2)**2)
            g.add_edge(src_node_id, dst_node_id, distance=euclidian_distance)

In [ ]:
g.nodes(data=True)

In [ ]:
g.edges(data=True)

In [ ]:
node_positions = {node_id: (x_values[node_id], y_values[node_id])
    for node_id in range(num_nodes)
}
nx.draw(g, pos=node_positions)

In [ ]:
# Convert the graph into PyTorch geometric
pyg_graph = from_networkx(g)
pyg_graph


In [ ]:
pyg_graph.edge_index

In [ ]:
pyg_graph.distance

In [ ]:
dict(nx.all_pairs_shortest_path(G=g))

In [ ]:
distance_matrix = nx.floyd_warshall_numpy(G=g, weight="distance")
distance_matrix


In [ ]:
distance_matrix = nx.floyd_warshall_numpy(G=g, weight="distance")
permutation, distance = solve_tsp_dynamic_programming(distance_matrix)
print(f"permutation: {permutation} - distance: {distance}")

In [ ]:
# draw graph
nx.draw(g, pos=node_positions)
# draw highlighted path
path_edges = [[permutation[i], permutation[i+1]] for i in range(len(permutation)-1)]
last_path_edge = [permutation[-1], permutation[0]]
path_edges.append(last_path_edge)
nx.draw_networkx_edges(G=g,pos=node_positions,edgelist=path_edges, edge_color ='r', width=10)

In [ ]:
# add solution to graph as a feature/attribute
solution_dict = {edge: 0 for edge in list(g.edges())}
for edge in path_edges:
    solution_dict[tuple(edge)] = 1
nx.set_edge_attributes(G=g, values=solution_dict, name="solution")
g.edges(data=True)

In [ ]:
# Convert the graph into PyTorch geometric
pyg_graph = from_networkx(g)
pyg_graph


In [ ]:
pyg_graph.solution


In [ ]:
# save nx graph in GML format
graph_filename = "temp_nx_graph.gml"
graph_filepath = "./data/" + graph_filename
nx.write_gml(G=g, path=graph_filepath)

In [ ]:
# read nx graph back from GML file
nx_graph_from_gml = nx.read_gml(path=graph_filepath, destringizer=int)

In [ ]:
# verify that graph read is the same that was written
g_edge_view = g.edges(data=True)
gml_g_edge_view = nx_graph_from_gml.edges(data=True)
str(g_edge_view) == str(gml_g_edge_view)